In [53]:
%load_ext autoreload
%autoreload 2

from utils.generate_constraints import Tmp

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
file_names_dict = {
    'supervised': ['L1', 'L2', 'L3'],
    'unsupervised': ['U'],
    'rule': ['rules']
}

data_dir_path = './inputs/toy_data'

In [70]:
tmp = Tmp(data_dir_path, file_names_dict)

In [71]:
tmp.load_data()


In [72]:
tmp.load_rules()

In [73]:
tmp.KB_origin

[['p_1(x)', '→', 'p_2(x)'], ['p_2(x)', '→', 'p_3(x)']]

In [74]:
tmp.KB

[[1, '∧', '¬', 'p_1(x)', '+', 'p_2(x)'],
 [1, '∧', '¬', 'p_2(x)', '+', 'p_3(x)']]